In [ ]:
# 安装需要的python packages

In [ ]:
!pip install -qU \
    openai==0.27.7 \
    "pinecone-client[grpc]"==2.2.1 \
    langchain==0.0.162 \
    tiktoken==0.4.0 \
    datasets==2.12.0

In [ ]:
# 使用的数据集是斯坦福 Question-Answering datasets

In [1]:
from datasets import load_dataset

data = load_dataset('squad', split='train')
data 

C:\Users\yanca\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset squad (C:/Users/yanca/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [ ]:
# 去重

In [2]:
data = data.to_pandas()
data.head()

data.drop_duplicates(subset='context', keep='first', inplace=True)
data.head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
5,5733bf84d058e614000b61be,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",When did the Scholastic Magazine of Notre dame...,"{'text': ['September 1876'], 'answer_start': [..."
10,5733bed24776f41900661188,University_of_Notre_Dame,The university is the major seat of the Congre...,Where is the headquarters of the Congregation ...,"{'text': ['Rome'], 'answer_start': [119]}"
15,5733a6424776f41900660f51,University_of_Notre_Dame,The College of Engineering was established in ...,How many BS level degrees are offered in the C...,"{'text': ['eight'], 'answer_start': [487]}"
20,5733a70c4776f41900660f64,University_of_Notre_Dame,All of Notre Dame's undergraduate students are...,What entity provides help with the management ...,"{'text': ['Learning Resource Center'], 'answer..."


In [ ]:
# 取少量数据来测试

In [3]:
data = data.head(20) # add for test

In [ ]:
# 通过langchain来初始化gpt-3.5-turbo model

In [27]:
# get a token: https://platform.openai.com/account/api-keys

from getpass import getpass

OPENAI_API_KEY = getpass()


 ········


In [26]:
from getpass import getpass
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'gpt-3.5-turbo-16k'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

In [33]:
import openai

openai.api_key = OPENAI_API_KEY

In [36]:
def get_embeddings_vector(string):
  # res = openai.Embedding.create(input=[string], engine=model_name)
  embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
    )
  res = embed.embed_documents(string)
  print(f'Generated embeddings for the string "{string[0:100]}", dimensions: {len(res["data"][0]["embedding"])}')
  return res["data"][0]["embedding"]

In [37]:
# test
query = "this is a test!"
get_embeddings_vector(query)

PermissionError: You are not allowed to generate embeddings from this model

In [ ]:
def get_vector_id(meta):
  vector_id_part1 = meta['date']
  vector_id_part2 = meta['url'].split('/')[-1]
  vector_id = f'{vector_id_part1}-{vector_id_part2}'
  print(f'vector_id = {vector_id}')
  return vector_id

In [ ]:
def save_vector_plus_meta(index, doc, embeds):
  try:
    text_embeddings = [(doc["text"], embeds)]
    metadatas = [doc]
    vector_id = get_vector_id(doc)
    ids = [vector_id]
    """
    def add_embeddings(text_embeddings: Iterable[Tuple[str, List[float]]], metadatas: Optional[List[dict]]=None, ids: Optional[List[str]]=None, **kwargs: Any) -> List[str]
    Run more texts through the embeddings and add to the vectorstore.

    Args:
      text_embeddings: Iterable pairs of string and embedding to
          add to the vectorstore.
      metadatas: Optional list of metadatas associated with the texts.
      ids: Optional list of unique IDs.

    Returns:
      List of ids from adding the texts into the vectorstore.
    """
    ids_added = index.add_embeddings(text_embeddings, metadatas, ids)
    print(f"Vectors added to FAISS DB - {len(ids_added)}: {ids_added}")
    return ids_added
  except Exception as e:
    print(f"[save_vector_plus_meta] exception of type {type(e).__name__}: {e}")
    if type(e).__name__ == "ValueError" and "Tried to add ids that already exist" in str(e):
      return
    else:
      raise e

In [ ]:
# 使用 Faiss 作为向量数据库，持久化存储docs

In [ ]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_size = 100

texts = []
metadatas = []

for i in tqdm(range(0, len(data), batch_size)):
    # get end of batch
    i_end = min(len(data), i+batch_size)
    batch = data.iloc[i:i_end]
    # first get metadata fields for this record
    metadatas = [{
        'title': record['title'],
        'text': record['context']
    } for j, record in batch.iterrows()]
    # get the list of contexts / documents
    documents = batch['context']
    # create document embeddings
    embeds = embed.embed_documents(documents)

    db = FAISS.from_documents(data, embeds)
    # # get IDs
    # ids = batch['id']
    # # add everything to pinecone
    # index.upsert(vectors=zip(ids, embeds, metadatas))

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(data, OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY))